In [1]:
library(rpart)
library(pROC)

# Define the directory structure using file.path()
my_dir <- file.path("C:/Users/GPEL_HPENVY/Desktop/Project_Predictive_Analytics_R")

# Check if the directory exists
if (dir.exists(my_dir)) {
  # Set the working directory
  setwd(my_dir)
  cat("Working directory set to:", getwd(), "\n")
} else {
  cat("Directory does not exist:", my_dir, "\n")
}

Warning message:
"package 'pROC' was built under R version 3.6.3"Type 'citation("pROC")' for a citation.

Attaching package: 'pROC'

The following objects are masked from 'package:stats':

    cov, smooth, var



Working directory set to: C:/Users/GPEL_HPENVY/Desktop/Project_Predictive_Analytics_R 


In [2]:
# set file path
# read csv file
#setwd('H:/GPEL_HPENVY/Documents/UNSW/07_ZZBU6511/Assessment3/')
employees <- read.csv(file = 'employees2.csv')

# Modify the column names to remove unwanted characters (assuming the problematic column is named "ï..Age")
colnames(employees) <- gsub("ï\\.{2}", "", colnames(employees))

In [3]:
# Inspect file
head(employees)


Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EnvironmentSatisfaction,Gender,JobInvolvement,JobRole,...,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,HighPerformance
26,No,Travel_Frequently,Research & Development,10,3,3,Male,4,Laboratory Technician,...,1,1,8,3,3,7,7,7,7,0
33,No,Travel_Rarely,Research & Development,13,5,4,Male,3,Healthcare Representative,...,1,0,14,6,3,0,0,0,0,0
59,No,Travel_Rarely,Research & Development,5,3,3,Female,4,Laboratory Technician,...,1,3,12,3,2,1,0,0,0,1
42,No,Travel_Rarely,Research & Development,21,3,2,Male,4,Healthcare Representative,...,3,2,10,2,2,0,0,0,0,0
24,No,Travel_Rarely,Research & Development,27,1,4,Female,2,Manufacturing Director,...,2,2,5,6,3,4,2,3,2,0
29,No,Travel_Rarely,Research & Development,37,2,3,Male,4,Research Scientist,...,3,1,10,3,3,10,7,0,7,1


In [4]:
# Add new binary variables to the employees data frame
employees$Is.Attrition = ifelse(employees$Attrition == "Yes", 1, 0)
employees$Over18 <- NULL  # Don't include Over18 since all response is Yes.

In [5]:
set.seed(57)
rows <- sample(nrow(employees), 0.8*nrow(employees))
training <- employees[rows, ]
testing <- employees[-rows, ]
# Create the models
logmod1 <- glm(
  formula = Is.Attrition ~ . - Attrition - Is.Attrition,
  data = training,
  family = binomial()
)

In [6]:
logmod2 <- glm(
  formula = Is.Attrition ~ BusinessTravel +  EnvironmentSatisfaction +
    JobInvolvement + JobSatisfaction + NumCompaniesWorked + OverTime,
  data = training,
  family = binomial()
)


In [7]:
logmod3 <- glm(
  formula = Is.Attrition ~ BusinessTravel +  EnvironmentSatisfaction +
    JobInvolvement + JobSatisfaction + OverTime,
  data = training,
  family = binomial()
)

In [8]:
# Get the probabilities on training and testing
training$logprob1 <- predict(logmod1, newdata=training, type="response")
testing$logprob1 <- predict( logmod1, newdata=testing, type="response")
training$logprob2 <- predict( logmod2, newdata=training, type="response")
testing$logprob2 <- predict( logmod2, newdata=testing, type="response")
training$logprob3 <- predict( logmod3, newdata=training, type="response")
testing$logprob3 <- predict( logmod3, newdata=testing, type="response")

In [9]:
# Get the ROC curves on training and testing
training.logroc1 <- roc(Is.Attrition ~ logprob1, data=training, plot=FALSE, quiet=TRUE
)
testing.logroc1 <- roc(Is.Attrition ~ logprob1, data=testing, plot=FALSE, quiet=TRUE
)
training.logroc2 <- roc(Is.Attrition ~ logprob2, data=training, plot=FALSE, quiet=TRUE
)
testing.logroc2 <- roc(Is.Attrition ~ logprob2, data=testing, plot=FALSE, quiet=TRUE
)
training.logroc3 <- roc(Is.Attrition ~ logprob3, data=training, plot=FALSE, quiet=TRUE
)
testing.logroc3 <- roc(Is.Attrition ~ logprob3, data=testing, plot=FALSE, quiet=TRUE
)

In [10]:
# Open a PNG graphics device
png("Fig6.1.5.png", width = 800, height = 600)  # Specify the file name and dimensions

# Plot the ROC curves and show AUC
par(mfrow=c(3, 2), cex.main=0.8)
plot(
  training.logroc1,
  main=paste("Model1: Log model on training set\nAUC =", round(training.logroc1$auc, digits=4))
)
plot(
  testing.logroc1,
  main=paste("Model1: Log model on testing set\nAUC =", round(testing.logroc1$auc, digits=4))
)
plot(
  training.logroc2,
  main=paste("Model2: Log model on training set\nAUC =", round(training.logroc2$auc, digits=4))
)
plot(
  testing.logroc2,
  main=paste("Model2: Log model on testing set\nAUC =", round(testing.logroc2$auc, digits=4))
)
plot(
  training.logroc3,
  main=paste("Model3: Log model on training set\nAUC =", round(training.logroc3$auc, digits=4))
)
plot(
  testing.logroc3,
  main=paste("Model3: Log model on testing set\nAUC =", round(testing.logroc3$auc, digits=4))
)
# Close the PNG graphics device
dev.off()

png 
  2